In [1]:
import boto3

In [2]:
BUCKET= 'andrew-ocr'

In [3]:
REGION = aws_region = boto3.session.Session().region_name # 'us-west-2'
s3 = boto3.client('s3', REGION)

In [4]:
output = s3.get_object(Bucket=BUCKET, Key='output-part1.json')

In [5]:
len(output)

9

In [6]:
output

{'ResponseMetadata': {'RequestId': 'FJTF0S2CEY8YE3FC',
  'HostId': 'nR5L2/TKgAEUg4JMfy21p6eQV8hJkynnYsth5+AxbIrPnNYfyLq7geMaXhzd6BLWhQiHyQZXhq0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'nR5L2/TKgAEUg4JMfy21p6eQV8hJkynnYsth5+AxbIrPnNYfyLq7geMaXhzd6BLWhQiHyQZXhq0=',
   'x-amz-request-id': 'FJTF0S2CEY8YE3FC',
   'date': 'Tue, 14 Sep 2021 20:45:41 GMT',
   'last-modified': 'Tue, 14 Sep 2021 18:18:40 GMT',
   'etag': '"ef2099051d378de4db54320030497df8"',
   'x-amz-version-id': 'sRB2SHzjGRIeMV3Xmxex6bOp1AgtrVof',
   'accept-ranges': 'bytes',
   'content-type': 'binary/octet-stream',
   'server': 'AmazonS3',
   'content-length': '70270073'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2021, 9, 14, 18, 18, 40, tzinfo=tzutc()),
 'ContentLength': 70270073,
 'ETag': '"ef2099051d378de4db54320030497df8"',
 'VersionId': 'sRB2SHzjGRIeMV3Xmxex6bOp1AgtrVof',
 'ContentType': 'binary/octet-stream',
 'Metadata': {},
 'Body': <botocore.response.Str

In [7]:
# import botocore.response as br
# streaming_obj = br.StreamingBody(raw_stream=None,content_length=5000)
streaming_obj = output['Body']
# streaming_obj.read()
string_data = streaming_obj.read().decode("utf-8")

In [11]:
json_data= json.loads(string_data)

In [13]:
len(json_data)

116

In [17]:
json_data[0].keys()

dict_keys(['DocumentMetadata', 'JobStatus', 'NextToken', 'Blocks', 'AnalyzeDocumentModelVersion', 'ResponseMetadata'])

In [60]:
[json_data[cnt]['DocumentMetadata'] for cnt in range(len(json_data))]

[{'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241},
 {'Pages': 241

In [ ]:
# string_data

In [35]:
len(json_data[0]['Blocks'])

1000

In [44]:

def get_csv(response):
    # Get the text blocks
    blocks=response['Blocks']

    blocks_map = {}
    table_blocks = []
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
        return "<b> NO Table FOUND </b>"

    csv = ''
    for index, table in enumerate(table_blocks):
        csv += generate_table_csv(table, blocks_map, index +1)
        csv += '\n\n'

    return csv

def generate_table_csv(table_result, blocks_map, table_index):
    rows = get_rows_columns_map(table_result, blocks_map)

    table_id = 'Table_' + str(table_index)
    
    # get cells.
    csv = 'Table: {0}\n\n'.format(table_id)

    for row_index, cols in rows.items():
        
        for col_index, text in cols.items():
            csv += '{}'.format(text) + ","
        csv += '\n'
        
    csv += '\n\n\n'
    return csv

def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}
                        
                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] =='SELECTED':
                            text +=  'X '    
    return text

In [45]:
tmp_csv= get_csv(json_data[0])

In [46]:
tmp_csv

"Table: Table_1\n\nComptroller ,Auditor ,Treasurer ,Comm. of Revenue ,Director of Public Welfare ,Director of Public Health ,Director of Public ,Dinctor of ,,\nsignity $ None dris ,$ None ,$ 8,650 3.000 ,$ 8,800 3,000 ,$ 5,400 ,$11,287.50 Neme ,Works $10,750 - ,$1400 -$7, ,Real ,\nagtion ,--- ,9,000 7.000 ,9,000 ,6,930 ,9,750 7.300 ,10,600 Director- $8,450 ments-$7,400; - Director of Public ,1450 Drector Dinctor of Inspections - Safety-$9.40. ,Finance Director. Real Estate Assess- $8,450; Personne) Libraries-$7,50 $8,450; Director t6 ,\n- - ,Clerk ,3,600 City's 2/3 ,3,814 City's 2/3 ,4,410 ,--- Public Wellare ,4,500 ,,,\nluma Vista ... ,--- ,3,432 ,3,000 ,3,000 ,- - ,3,060 Supt. ,,Supt. Water 1 Sewer-$3,060, ,\n... ,--- ,6,800 ,5,500 ,4,500 ,9,500 ,... ,4,120 ,Director of Finance & Clerk of Council- ,\n,,9.900 ,9,900 ,10,408 ,,$6,500; ,Plannisg.bgineer-$6,000. ,- ,\n13.440 Parille --- ,9.360 --- ,2,533 City's 1/3 ,5,850 City's 2/3 ,4,800 ,9,000 ,8,200 ,1,00 ,Director of Finance & Clerk

In [47]:
tmp_csv1= get_csv(json_data[1])
tmp_csv1

'Table: Table_1\n\nMunicipality ,Mayor ,Town Manager ,Councilman ,Town Clerk ,Town Attorney ,Fire Chief ,\nCape Charles (2,427) Cedar Bluff (1, ,083) Charlotte C.H. (397) ,$ 600 --- --- ,$--- --- --- ,$ 300 --- --- ,$ 1,200 - also - treasurer 1,800 - also treasurer. 1,080 ,$ 600 --- --- ,$ 100 --- --- ,\nChase City ,900 ,--- ,100 ,3,816 ,50 ,--- ,\n(2,519) ,,,,,retainer ,,\nChatham ,500 ,--- ,60 ,--- ,--- ,.-- ,\n(1,760) Chilhowie (1,022) ,--- 60 ,--- ,--- ,1,800 ,100 ,--- ,\nChincoteague ,600 ,--- ,180 ,1,764 ,400 ,--- ,\n(3,724) Christiansburg (2,967) ,480 ,6,600 ,120 ,1,050 ,300 ,--- ,\nCrewe (2,030) ,1,080 ,4,800 ,60 ,3,240 ,300 retainer ,90 ,\nCulpeper (2,527) ,300 ,7,800 ,300 ,1,200 ,280 ,Volunteer ,\nDamascus ,840 ,--- ,60 - 90 ,360 ,--- ,--- ,\n(1,726) Dayton ,240 ,--- ,5 ,1,250 also ,250 ,--- ,\n(788) Dublin ,360 ,--- 000 ,24 ,--- ,Fees ,--- ,\n(1,313) Edinburg (533) Emporia ,100 300 ,--- 6,000 ,36 --- 150 ,750 - also treasurer 3,000 ,--- 1,200 ,--- --- ,\n(5,664) Fairfax (1,9

In [74]:
all_blocks= []
for cnt in range(len(json_data)):
    block= json_data[cnt]['Blocks']
    all_blocks.extend(block)
final_dict= {'Blocks': all_blocks}

In [87]:
tmp_csv= get_csv(final_dict)

prefix= 'csvs/'
output_file = prefix+ 'ALL_tables' +'.csv'
# output_file = prefix+ 'table_'+ str(cnt) +'.csv'

# replace content
with open(output_file, "wt") as fout:
    fout.write(tmp_csv)

In [76]:
len(tmp_csv)

299677

In [78]:
tmp_csv

'Table: Table_1\n\nComptroller ,Auditor ,Treasurer ,Comm. of Revenue ,Director of Public Welfare ,Director of Public Health ,Director of Public ,Dinctor of ,,\nsignity $ None dris ,$ None ,$ 8,650 3.000 ,$ 8,800 3,000 ,$ 5,400 ,$11,287.50 Neme ,Works $10,750 - ,$1400 -$7, ,Real ,\nagtion ,--- ,9,000 7.000 ,9,000 ,6,930 ,9,750 7.300 ,10,600 Director- $8,450 ments-$7,400; - Director of Public ,1450 Drector Dinctor of Inspections - Safety-$9.40. ,Finance Director. Real Estate Assess- $8,450; Personne) Libraries-$7,50 $8,450; Director t6 ,\n- - ,Clerk ,3,600 City\'s 2/3 ,3,814 City\'s 2/3 ,4,410 ,--- Public Wellare ,4,500 ,,,\nluma Vista ... ,--- ,3,432 ,3,000 ,3,000 ,- - ,3,060 Supt. ,,Supt. Water 1 Sewer-$3,060, ,\n... ,--- ,6,800 ,5,500 ,4,500 ,9,500 ,... ,4,120 ,Director of Finance & Clerk of Council- ,\n,,9.900 ,9,900 ,10,408 ,,$6,500; ,Plannisg.bgineer-$6,000. ,- ,\n13.440 Parille --- ,9.360 --- ,2,533 City\'s 1/3 ,5,850 City\'s 2/3 ,4,800 ,9,000 ,8,200 ,1,00 ,Director of Finance & C

In [81]:
len(final_dict['Blocks'])

115834

In [84]:
final_dict.keys()

dict_keys(['Blocks'])

In [86]:
cnt=0 
blocks= final_dict['Blocks']
table_blocks = []
for block in blocks:
    if block['BlockType'] == "TABLE":
        table_blocks.append(block)
        cnt+= 1
print(cnt)

202


In [73]:
json_data[0]['Blocks'][3]

{'BlockType': 'LINE',
 'Confidence': 98.1883316040039,
 'Text': '-3-',
 'Geometry': {'BoundingBox': {'Width': 0.013985559344291687,
   'Height': 0.021288499236106873,
   'Left': 0.08516582101583481,
   'Top': 0.4616546630859375},
  'Polygon': [{'X': 0.08516582101583481, 'Y': 0.48260051012039185},
   {'X': 0.0854455977678299, 'Y': 0.4616546630859375},
   {'X': 0.0991513803601265, 'Y': 0.46196678280830383},
   {'X': 0.09887868911027908, 'Y': 0.48294317722320557}]},
 'Id': '4bbb078a-94c6-4c46-95f0-613411a76ab3',
 'Relationships': [{'Type': 'CHILD',
   'Ids': ['828c327a-8bf9-4552-a5eb-e247b5d94b4e']}],
 'Page': 1}

In [67]:
json_data[0]['NextToken']

'mZEttZNZj0yBDvf+PqnVRkH7IbjaezVAZMfzAZQtU1GhViF/r05C7Rub2RKYi1B49xM7C52FCzWv0BMo0sJYerpAkSpMFaPrX8xhLPXZxCt5HfZ1ng=='

In [66]:
[json_data[cnt]['NextToken'] for cnt in range(len(json_data)) ]
# if 'NexToken' in json_data[cnt]

KeyError: 'NextToken'

In [79]:
# for cnt in range(len(json_data)):
#     tmp_csv= get_csv(json_data[cnt])
    
#     prefix= 'csvs/'
#     output_file = prefix+ 'table_'+ str(cnt) +'.csv'

#     # replace content
#     with open(output_file, "wt") as fout:
#         fout.write(tmp_csv)

In [55]:
len(json_data)

116

In [57]:
json_data[2].keys()

dict_keys(['DocumentMetadata', 'JobStatus', 'NextToken', 'Blocks', 'AnalyzeDocumentModelVersion', 'ResponseMetadata'])

In [58]:
json_data[2]['NextToken']

'mZEttZNZj0yBDvf+PqnVRkH7IbjaezVAZMfzAZQtU1GhViF/r05C7Rub2RKYi1B49xM7C52FCzWv0BMo0sJYerpAkSpMFaPrX8xhLPTfzSt5H/Z1ng=='